# Innoplexus Online Hiring Hackathon: Sentiment Analysis

- https://datahack.analyticsvidhya.com/contest/innoplexus-online-hiring-hackathon/
- https://www.machinelearningplus.com/nlp/gensim-tutorial/  *Check out this tutorial on gensim*  

# Problem Statement


## Sentiment Analysis for drugs/medicines

Nowadays the narrative of a brand is not only built and controlled by the company that owns the brand. For this reason, companies are constantly looking out across Blogs, Forums, and other social media platforms, etc for checking the sentiment for their various products and also competitor products to learn how their brand resonates in the market. This kind of analysis helps them as part of their post-launch market research. This is relevant for a lot of industries including pharma and their drugs.
 

The challenge is that the language used in this type of content is not strictly grammatically correct. Some use sarcasm. Others cover several topics with different sentiments in one post. Other users post comments and reply and thereby indicating his/her sentiment around the topic.

Sentiment can be clubbed into 3 major buckets - Positive, Negative and Neutral Sentiments.

 

You are provided with data containing samples of text. This text can contain one or more drug mentions. Each row contains a unique combination of the text and the drug mention. Note that the same text can also have different sentiment for a different drug.

Given the text and drug name, the task is to predict the sentiment for texts contained in the test dataset. Given below is an example of text from the dataset:

 

Example:

*Stelara is still fairly new to Crohn's treatment. This is why you might not get a lot of replies. I've done some research, but most of the "time to work" answers are from Psoriasis boards. For Psoriasis, it seems to be about 4-12 weeks to reach a strong therapeutic level. The good news is, Stelara seems to be getting rave reviews from Crohn's patients. It seems to be the best med to come along since Remicade. I hope you have good success with it. My daughter was diagnosed Feb. 19/07, (13 yrs. old at the time of diagnosis), with Crohn's of the Terminal Illium. Has used Prednisone and Pentasa. Started Imuran (02/09), had an abdominal abscess (12/08). 2cm of Stricture. Started ​Remicade in Feb. 2014, along with 100mgs. of Imuran.*

For Stelara the above text is ​positive​ while for Remicade the above text is ​negative​.

*TRAIN DATA*  
unique_hash - 	Unique ID  
text	    -   text pertaining to the drugs  
drug	    -   drug name for which the sentiment is provided  
sentiment	-   (Target) 0-positive, 1-negative, 2-neutral  

### THINGS TO DO FOR IMPROVING MODEL

*2 sentiment remove few records  
fasttext  
drug column...feature add 200, train word 2 vec  
random forrest  
gbbost  
deep learning -lstm  
hoole house andrew ng  
analysis on sentiment - give weightage....multiply buy 1.5 every weight*

*pca  
tfidf change  
ensemble  
 predictprobba 3 class probability -  1 row  
 append all 3   
 y train shape same  
 x_Train 9 cols  
 logistic regression new dataset on 10   
 test data same process*  
 

In [2]:
# more common imports
import pandas as pd
import numpy as np
from collections import Counter
import re

# languange processing imports
import nltk 
from nltk.corpus import stopwords
from textblob import Word
from gensim.corpora import Dictionary

# preprocessing imports
from sklearn.preprocessing import LabelEncoder

# model for word2vec
from gensim.models.word2vec import Word2Vec

#model for vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#model for splitting of data
from sklearn.model_selection import train_test_split

#logistic regression algo
from sklearn.linear_model import LogisticRegression

#naive bayes
from sklearn.naive_bayes import MultinomialNB

#pca for dimension reduction
from sklearn.decomposition import RandomizedPCA

#xgboost
# from xgboost import XGBClassifier

#svm algo 
from sklearn import svm

# Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import fbeta_score, accuracy_score
from sklearn.metrics import classification_report

# hyperparameter training imports
from sklearn.model_selection import GridSearchCV

# visualization imports
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import base64
import io
%matplotlib inline
sns.set()  # defines the style of the plots to be seaborn style

##pretty print
import pprint as pp

#punctuation strings
import string

#logging each run of cells
#import logging

#use multi cores
import multiprocessing

#calculation of time
from time import time

#accuracy of model
from sklearn.metrics import accuracy_score

C:\Users\tshrs\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
pd.set_option('max_colwidth', -1)

In [5]:
train_data = pd.read_csv('train.csv')
print(train_data.shape)
train_data.head()

(5279, 4)


,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,"Autoimmune diseases tend to come in clusters. As for Gilenya – if you feel good, don’t think about it, it won’t change anything but waste your time and energy. I’m taking Tysabri and feel amazing, no symptoms (other than dodgy color vision, but I’ve had it since always, so, don’t know) and I don’t know if it will last a month, a year, a decade, ive just decided to enjoy the ride, no point in worrying.",gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,"I can completely understand why you’d want to try it. But, results reported in lectures don’t always stand up to the scrutiny of peer-review during publication. There so much still to do before this is convincing. I hope that it does work out, I really do. And if you’re aware of and happy with the risks, then that’s great. I just think it’s important to present this in a balanced way, and to understand why we don’t move straight from the first show of promise in an animal study to using drugs on humans. There’s still a lot of animal data to gather, and human data to gather before anyone can tell if it’s safe or effective. I can’t tell you how many times animal studies don’t follow through to humans, but it’s one of the major attrition points in drug development. You’ve been through some of the unpredictability issues with Cladribine/Gilenya, where there was an interaction that wasn’t predicted. But once people try it, the doctors can see patterns and work out what’s going on. Clemastine/metformin is very exciting, and given what you’ve said about your current condition and your personal risk tolerance it makes sense to try it. It definitely wouldn’t be for everyone.",gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,"Interesting that it only targets S1P-1/5 receptors rather than 1-5 like Fingolimod. Hoping to soon see what the AEs and SAEs were Yes. I'm not sure what this means, exactly: Quote Nine patients reported serious adverse events (2 mg: 3/29 [10.3%], 1.25 mg: 1/43 [2.3%], 0.5 mg: 4/29 [13.8%], and 0.25 mg: 1/50 [2.0%]; no serious adverse event was reported for more than 1 patient and no new safety signals occurred compared with the BOLD Study. If there were 9 patients reporting SAEs, how can it be stated that ""no serious adverse event was reported for more than 1 patient...""? Maybe I haven't read this right, or maybe there's a misprint. I'm very pleased that something is being developed for SPMS, and it's encouraging that siponimod doesn't linger for very long in the body.",fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder where lemtrada and ocrevus sales would go, if they prove anti-cd20 are induction",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain and Cervical Cord are in and my next Neurologist appointment is in the next couple of weeks. There’re no new lesions in Brain/Cord and I’ve had no relapses while I was on Gilenya. This was a good sign. But there was one line in the cervical cord review that concerned me. It goes : “Lesions at C2-3 and T2 now show hypointensity on the post gadolinium T1 images only. This could represent artifact or early axonal loss.” That was bothersome to read. What are the kind of symptoms from C2-C3 lesion should I be aware of ? Would it result in change of my DMT ? Thanks.",gilenya,1


In [6]:
test_data = pd.read_csv('test.csv')
print(test_data.shape)
test_data.head(1)

(2924, 3)


,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 55% switching to fingolimod",fingolimod


### Inspect Sentiment variable

In [7]:
train_data.sentiment.value_counts()

2    3825
1    837 
0    617 
Name: sentiment, dtype: int64

In [8]:
# train_data.drug.value_counts()

In [9]:
train_test_list = [ train_data, test_data ]

#here ignore_index True will create new values from 0 to n-1
train_test_data = pd.concat(train_test_list, ignore_index=True)

C:\Users\tshrs\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [10]:
train_test_data.shape

(8203, 4)

# Text pre-processing

In [11]:
def text_preprocessing(data):
    
    #convert text to lower-case
    data['tokenized_text'] = data['text'].apply(lambda x:' '.join(x.lower() for x in x.split()))

    #remove punctuations, unwanted characters
    data['tokenized_text_1']= data['tokenized_text'].apply(lambda x: "".join([char for char in x if char not in string.punctuation]))

    #remove numbers
    data['tokenized_text_2']= data['tokenized_text_1'].apply(lambda x: re.sub('[0-9]+', ' ' , x))

    #remove stopwords
    stop = stopwords.words('english')
    data['tokenized_text_3']= data['tokenized_text_2'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))

    #lemmatization
    data['tokenized_text_4']= data['tokenized_text_3'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

#         # remove all single characters
#     document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    #create a final text field to work on
    data['final_text'] = data['tokenized_text_4']

# Vectorize

In [12]:
#implementing tfidf
1. # max_df = ignore terms that appear in more than 50% of the documents

2. # min_df = only keep terms that appear in at least 2 documents

#          If float, the parameter represents a proportion of documents.
#          If integer, the parameter represents an absolute count.          

3. # # ngram_range # #
# Example: range(1,2)
# v=text.CountVectorizer(ngramrange=(1,2)) print(v.fit(["an apple a day keeps the doctor away"]).vocabulary)

# Result:
# {'an': 0, 'apple': 2, 'day': 5, 'keeps': 9, 'the': 11, 'doctor': 7, 'away': 4, 'an apple': 1, 'apple day': 3, 'day keeps': 6, 'keeps the': 10, 'the doctor': 12, 'doctor away': 8}

# Example: range(1,3)
# v=text.CountVectorizer(ngramrange=(1,3)) print(v.fit(["an apple a day keeps the doctor away"]).vocabulary)

# Result:
# {'an': 0, 'apple': 3, 'day': 7, 'keeps': 12, 'the': 15, 'doctor': 10, 'away': 6, 'an apple': 1, 'apple day': 4, 'day keeps': 8, 'keeps the': 13, 'the doctor': 16, 'doctor away': 11, 'an apple day': 2, 'apple day keeps': 5, 'day keeps the': 9, 'keeps the doctor': 14, 'the doctor away': 17}


def vectorization(data):

#     tf=TfidfVectorizer( min_df=10 
#                        , max_df=0.7
#                        , lowercase=True 
#                        , ngram_range=(0, 2) )
    t0 = time()
    tf=TfidfVectorizer( min_df=10  
                       , ngram_range=(1, 2) )

    tfidf_dtm = tf.fit_transform(data).toarray()
    print('Shape of document term matrix : ', tfidf_dtm.shape)
    print("done in %0.3fs" % (time() - t0))
    return list(tfidf_dtm) , tf

In [16]:
#label the drug column

def merge_drug_tfidf(data):
    
    label_encoder = LabelEncoder()
    label_encoder.fit(data.drug)
    data['drug_label'] = label_encoder.transform(data.drug)

    # here we add the drug column label to the tfidf vector

    ls = []
    for key,value in data.iterrows():
        ls.append(np.append(value['tfidf'],value['drug_label']))

    return ls 

In [14]:

#pre-processing or cleaning data

text_preprocessing(train_data)
text_preprocessing(train_test_data)


In [53]:
#build a vector from the txt data

train_data['tfidf'] , tf = vectorization(train_data['final_text'])
#train_test_data['tfidf'] , tf = vectorization(train_test_data['final_text'])

Shape of document term matrix :  (5279, 13951)
done in 5.537s


In [19]:
#merge the drug column with the tfidf vector, append at last

train_data['final'] = merge_drug_tfidf(train_data)
#train_test_data['final'] = merge_drug_tfidf(train_test_data)

# 3- Classsification

In [ ]:
# from imblearn.over_sampling import SMOTE

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)

# print("Number transactions X_train dataset: ", X_train.shape)
# print("Number transactions y_train dataset: ", X_val.shape)
# print("Number transactions X_test dataset: ", X_test.shape)
# print("Number transactions y_test dataset: ", y_val.shape)

In [61]:
# print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
# print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

# sm = SMOTE(random_state=2)
# X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

# print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
# print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

# print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
# print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


In [25]:
def split_data(data , label):
    
    data_np = np.array(list(map(np.array, data)))    
    X_train, X_val, y_train, y_val = train_test_split(data_np, label, random_state=42 , test_size=0.20)

    print(X_train.shape)
    print(X_val.shape)
    print(y_train.shape)
    print(y_val.shape)    
    return X_train, X_val, y_train, y_val

In [35]:

#X_train, X_val, y_train, y_val = split_data(train_data['final'] , train_data['sentiment'])
X_train, X_val, y_train, y_val = split_data(train_data['final'] , train_data['sentiment'])

(4223, 13952)
(1056, 13952)
(4223,)
(1056,)


In [36]:
y_train.shape

(4223,)

In [64]:

# initiaize supervised learning models

clf_A = svm.SVC(kernel='linear'
                , class_weight='balanced'  # penalize
                , probability=True)
clf_B = LogisticRegression(random_state=42)
clf_C = MultinomialNB()
#clf_D = XGBClassifier()

#  Calculate the number of samples for 1%, 10%, and 100% of the training data
#  samples_100 is the entire training set i.e. len(y_train)
#  samples_10 is 10% of samples_100
#  samples_1 is 1% of samples_100

samples_100 = len(y_train)
samples_10 = int(len(y_train)*10/100)
samples_1 = int(len(y_train)/100)

#sentiment target names
target_names = ['0', '1' , '2' ]

print('samples_100 : ', samples_100)
print('samples_10  : ', samples_10)
print('samples_1   : ', samples_1)

samples_100 :  4223
samples_10  :  422
samples_1   :  42


In [33]:
def train_predict(learner, sample_size, X_train, y_train, X_val, y_val): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_val: features validation set
       - y_val: income validation set
    '''
    
    results = {}
    
    # Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = time() # Get end time
    
    # Calculate the training time
    results['train_time'] = end-start
        
    # Get the predictions on the validation set(X_val),
    start = time() # Get start time
    predictions_val = learner.predict(X_val)
    end = time() # Get end time
    
    # Calculate the total prediction time
    results['pred_time'] = end-start
        
    #Compute accuracy on test set using accuracy_score()
    results['acc_test'] = accuracy_score(y_val,predictions_val)
    
    #classification report for precision, relcall, f1-score
    results['f_val'] = classification_report(y_val, predictions_val, target_names=target_names)
               
    # Success
    print("{} trained on {} samples in {} time with prediction time {} .".format(learner.__class__.__name__
                                                                                , sample_size,results['train_time'] 
                                                                                , results['pred_time'] ))
    print()
    print("{} Classification Report ".format(learner.__class__.__name__) )
    print()
    print(results['f_val'] )
    print("Accuracy : ", results['acc_test'])
    print("******************************************************************************************************************")
        
    # Return the results
    return results

In [37]:
results = {}
for clf in [clf_A,clf_B ]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1 , samples_10, samples_100]):
        results[clf_name][i] = train_predict(clf, samples, X_train, y_train, X_val, y_val)

C:\Users\tshrs\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SVC trained on 65 samples in 1.793879747390747 time with prediction time 1.7457211017608643 .

SVC Classification Report 

             precision    recall  f1-score   support

          0       0.15      0.02      0.03       111
          1       0.00      0.00      0.00       175
          2       0.73      0.99      0.84       770

avg / total       0.55      0.72      0.62      1056

Accuracy :  0.7225378787878788
******************************************************************************************************************
SVC trained on 656 samples in 99.42350816726685 time with prediction time 17.306137084960938 .

SVC Classification Report 

             precision    recall  f1-score   support

          0       0.26      0.09      0.13       111
          1       0.40      0.28      0.33       175
          2       0.77      0.89      0.83       770

avg / total       0.65      0.71      0.67      1056

Accuracy :  0.7073863636363636
****************************************

In [65]:
results = {}
for clf in [clf_C ]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1 , samples_10, samples_100]):
        results[clf_name][i] = train_predict(clf, samples, X_train, y_train, X_val, y_val)

MultinomialNB trained on 42 samples in 0.017952919006347656 time with prediction time 0.13363957405090332 .

MultinomialNB Classification Report 

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       111
          1       0.00      0.00      0.00       175
          2       0.73      1.00      0.84       770

avg / total       0.53      0.73      0.61      1056

Accuracy :  0.7291666666666666
******************************************************************************************************************
MultinomialNB trained on 422 samples in 0.024934768676757812 time with prediction time 0.032911062240600586 .

MultinomialNB Classification Report 

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       111
          1       0.00      0.00      0.00       175
          2       0.73      0.99      0.84       770

avg / total       0.53      0.72      0.61      1056

Accuracy :  0.72443181

# 4. Training on full train set

In [48]:
X_train, X_val, y_train, y_val = split_data(train_data['final'] , train_data['sentiment'])

(4223, 13952)
(1056, 13952)
(4223,)
(1056,)


In [60]:
t = time()
trainer_model_svm = clf_A.fit(X_train,y_train)
print("Time taken to train the SVM test data : ", time()-t)

Time taken to train the SVM test data :  2069.0106766223907


In [49]:
t = time()
trainer_model_lr  = clf_B.fit(X_train,y_train)
print("Time taken to train the Logistic Regresssion test data : ", time()-t)

Time taken to train the Logistic Regresssion test data :  0.8393814563751221


# 5. Prediction on Test data

In [39]:
#here we do the same text processing as done for training data

text_preprocessing(test_data) #formats and cleans the data


In [54]:

test_data['tfidf'] = list(tf.transform(test_data['final_text']).toarray())
tf.transform(test_data['final_text']).toarray().shape

(2924, 13951)

In [55]:
#label the drug column

test_data['final'] = merge_drug_tfidf(test_data)

In [56]:
# create a numpy array for prediction

X_test        = np.array(list(map(np.array, test_data['final'])))

In [61]:
# predict from the choosen model

submission_predictions_svm = trainer_model_svm.predict(X_test)

In [57]:
submission_predictions_lr = trainer_model_lr.predict(X_test)

# 6. Submission

In [58]:
sample_sub = pd.read_csv('sample_submission.csv')
print(sample_sub.shape)
sample_sub.head()

(2924, 2)


,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,0
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,0
2,50b6d851bcff4f35afe354937949e9948975adf7,0
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,0
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,0


In [62]:
submission = pd.DataFrame({
        "unique_hash": test_data["unique_hash"],
        "sentiment": submission_predictions_svm
    })
submission.groupby('sentiment').count()

,unique_hash
sentiment,
0,436
1,542
2,1946


In [63]:
submission.to_csv('submission6_svm.csv', index=False)
#0.58289